# Video Game Sales Data Analytics Project

## Introduction

This project analyzes historical video game sales data to uncover strategic market insights for game publishers and industry stakeholders. 

The dataset contains detailed sales information across multiple platforms, genres, regions, and publishers, spanning several decades. Through SQL queries executed within a Python notebook, this project explores platform dominance, genre profitability, publisher performance, regional preferences, and temporal trends. The goal is to provide actionable business insights that can guide game development, marketing strategies, and investment decisions.

## Project Context

The video game industry is a dynamic and highly competitive market with diverse platforms and genres appealing to various regional audiences. Understanding sales trends and market behaviors is critical for publishers aiming to maximize revenue and allocate resources effectively. This dataset offers a rich foundation for data-driven analysis, representing millions of dollars in sales across North America, Europe, Japan, and other global markets.

## Problem Statement

You are tasked with answering key questions such as:

- Which platforms and genres dominate global and regional sales?
- How do sales trends evolve over the years, and what seasonal patterns exist?
- Which publishers lead the market, and what attributes correlate with higher sales?
- How do geographic regions differ in game preferences?
- What emerging opportunities exist in niche genres or underrepresented platforms?

## Key Objectives

- Load and clean the data for SQL analysis
- Perform exploratory data analysis to understand trends and outliers
- Generate aggregate and comparative metrics using SQL
- Employ window functions and complex queries for in-depth insights
- Summarize findings in a clear, business-oriented manner

By completing this project, you will develop practical skills in SQL-driven data analytics, learn how to present data-backed recommendations, and build a portfolio-quality project grounded in a real-world dataset.


In [5]:
import sqlite3
import pandas as pd

# Load the CSV dataset (update filename/path as needed)
csv_file = '../input/video-game-sales/vgsales.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file)

# Connect to in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [6]:
# Create table schema aligning to dataset description
cursor.execute('''
CREATE TABLE video_game_sales (
    Rank INTEGER PRIMARY KEY,
    Name TEXT,
    Platform TEXT,
    Year INTEGER,
    Genre TEXT,
    Publisher TEXT,
    NA_Sales FLOAT,
    EU_Sales FLOAT,
    JP_Sales FLOAT,
    Other_Sales FLOAT,
    Global_Sales FLOAT
)
''')

df.to_sql('video_game_sales', conn, if_exists='append', index=False)

# Helper function to run and display queries cleanly
def run_query(query, title=None):
    if title:
        print(f"## {title}\n")
    print(f"``````\n")
    result = pd.read_sql_query(query, conn)
    display(result.head(10))
    print('\n---\n')

# QUERY 1: Top Platforms by Global Sales and Regional Breakdown

In [7]:
query1 = """
SELECT Platform,
       ROUND(SUM(Global_Sales), 2) AS Total_Global_Sales,
       ROUND(SUM(NA_Sales), 2) AS NA_Sales,
       ROUND(SUM(EU_Sales), 2) AS EU_Sales,
       ROUND(SUM(JP_Sales), 2) AS JP_Sales,
       ROUND(SUM(Other_Sales), 2) AS Other_Region_Sales,
       COUNT(*) AS Number_Of_Games
FROM video_game_sales
GROUP BY Platform
ORDER BY Total_Global_Sales DESC;
"""
run_query(query1, "Top Platforms by Global Sales and Regional Breakdown")


## Top Platforms by Global Sales and Regional Breakdown

``````



,Platform,Total_Global_Sales,NA_Sales,EU_Sales,JP_Sales,Other_Region_Sales,Number_Of_Games
0,PS2,1255.64,583.84,339.29,139.20,193.44,2161
1,X360,979.96,601.05,280.58,12.43,85.54,1265
2,PS3,957.84,392.26,343.71,79.99,141.93,1329
3,Wii,926.71,507.71,268.38,69.35,80.61,1325
4,DS,822.49,390.71,194.65,175.57,60.53,2163
5,PS,730.66,336.51,213.60,139.82,40.91,1196
6,GBA,318.50,187.54,75.25,47.33,7.73,822
7,PSP,296.28,108.99,68.25,76.79,42.19,1213
8,PS4,278.10,96.80,123.70,14.30,43.36,336
9,PC,258.82,93.28,139.68,0.17,24.86,960



---



# QUERY 2: Most Profitable Genres Globally and Regionally

In [8]:
query2 = """
SELECT Genre,
       COUNT(*) AS Num_Games,
       ROUND(SUM(Global_Sales), 2) AS Total_Global_Sales,
       ROUND(AVG(Global_Sales), 2) AS Avg_Sales_Per_Game,
       ROUND(SUM(NA_Sales), 2) AS NA_Sales,
       ROUND(SUM(EU_Sales), 2) AS EU_Sales,
       ROUND(SUM(JP_Sales), 2) AS JP_Sales
FROM video_game_sales
GROUP BY Genre
ORDER BY Total_Global_Sales DESC;
"""
run_query(query2, "Most Profitable Genres Globally and Regionally")

## Most Profitable Genres Globally and Regionally

``````



,Genre,Num_Games,Total_Global_Sales,Avg_Sales_Per_Game,NA_Sales,EU_Sales,JP_Sales
0,Action,3316,1751.18,0.53,877.83,525.00,159.95
1,Sports,2346,1330.93,0.57,683.35,376.85,135.37
2,Shooter,1310,1037.37,0.79,582.60,313.27,38.28
3,Role-Playing,1488,927.37,0.62,327.28,188.06,352.31
4,Platform,886,831.37,0.94,447.05,201.63,130.77
5,Misc,1739,809.96,0.47,410.24,215.98,107.76
6,Racing,1249,732.04,0.59,359.42,238.39,56.69
7,Fighting,848,448.91,0.53,223.59,101.32,87.35
8,Simulation,867,392.20,0.45,183.31,113.38,63.70
9,Puzzle,582,244.95,0.42,123.78,50.78,57.31



---



# QUERY 3: Top Publishers by Total Sales and Number of Games

In [9]:

query3 = """
SELECT Publisher,
       COUNT(*) AS Num_Games,
       ROUND(SUM(Global_Sales), 2) AS Total_Global_Sales,
       ROUND(AVG(Global_Sales), 2) AS Avg_Sales_Per_Game
FROM video_game_sales
GROUP BY Publisher
HAVING Num_Games >= 5
ORDER BY Total_Global_Sales DESC
LIMIT 10;
"""
run_query(query3, "Top Publishers by Total Sales and Number of Games")


## Top Publishers by Total Sales and Number of Games

``````



,Publisher,Num_Games,Total_Global_Sales,Avg_Sales_Per_Game
0,Nintendo,703,1786.56,2.54
1,Electronic Arts,1351,1110.32,0.82
2,Activision,975,727.46,0.75
3,Sony Computer Entertainment,683,607.50,0.89
4,Ubisoft,921,474.72,0.52
5,Take-Two Interactive,413,399.54,0.97
6,THQ,715,340.77,0.48
7,Konami Digital Entertainment,832,283.64,0.34
8,Sega,639,272.99,0.43
9,Namco Bandai Games,932,254.09,0.27



---



# QUERY 4: Year-wise Trend of Games Released and Total Sales 

In [10]:

query4 = """
SELECT Year,
       COUNT(*) AS Games_Released,
       ROUND(SUM(Global_Sales), 2) AS Total_Global_Sales,
       ROUND(AVG(Global_Sales), 2) AS Avg_Sales_Per_Game
FROM video_game_sales
WHERE Year IS NOT NULL
GROUP BY Year
ORDER BY Year;
"""
run_query(query4, "Year-wise Trend of Games Released and Total Sales")

## Year-wise Trend of Games Released and Total Sales

``````



,Year,Games_Released,Total_Global_Sales,Avg_Sales_Per_Game
0,1980,9,11.38,1.26
1,1981,46,35.77,0.78
2,1982,36,28.86,0.80
3,1983,17,16.79,0.99
4,1984,14,50.36,3.60
5,1985,14,53.94,3.85
6,1986,21,37.07,1.77
7,1987,16,21.74,1.36
8,1988,15,47.22,3.15
9,1989,17,73.45,4.32



---



# QUERY 5: Regional Sales Summary 

In [11]:

query5 = """
SELECT
    ROUND(SUM(NA_Sales), 2) AS Total_NA_Sales,
    ROUND(SUM(EU_Sales), 2) AS Total_EU_Sales,
    ROUND(SUM(JP_Sales), 2) AS Total_JP_Sales,
    ROUND(SUM(Other_Sales), 2) AS Total_Other_Sales,
    ROUND(SUM(Global_Sales), 2) AS Total_Global_Sales
FROM video_game_sales;
"""
run_query(query5, "Regional Sales Summary")

## Regional Sales Summary

``````



,Total_NA_Sales,Total_EU_Sales,Total_JP_Sales,Total_Other_Sales,Total_Global_Sales
0,4392.95,2434.13,1291.02,797.75,8920.44



---



## Which platforms have shown consistent sales growth over multiple decades, and how does their market share evolve?

In [12]:
query6 = """
WITH DecadeSales AS (
  SELECT
    Platform,
    CASE 
      WHEN Year BETWEEN 1980 AND 1989 THEN '1980s'
      WHEN Year BETWEEN 1990 AND 1999 THEN '1990s'
      WHEN Year BETWEEN 2000 AND 2009 THEN '2000s'
      WHEN Year BETWEEN 2010 AND 2016 THEN '2010s'
      ELSE 'Other'
    END AS Decade,
    SUM(Global_Sales) AS Sales_Per_Decade
  FROM video_game_sales
  GROUP BY Platform, Decade
)
SELECT Platform, Decade, Sales_Per_Decade
FROM DecadeSales
ORDER BY Platform, Decade;
"""
run_query(query6, "Regional Sales Summary")


## Regional Sales Summary

``````



,Platform,Decade,Sales_Per_Decade
0,2600,1980s,86.57
1,2600,Other,10.51
2,3DO,1990s,0.10
3,3DS,2010s,246.28
4,3DS,Other,1.18
5,DC,1990s,8.56
6,DC,2000s,7.41
7,DS,1980s,0.02
8,DS,2000s,689.25
9,DS,2010s,129.40



---



## What genres have the highest blockbuster hit rate (games with sales > 1 million), and do these hits cluster around certain publishers or platforms?
Answer approach:
Identify top-selling games, calculate hit rates by genre, then cross-reference publisher and platform distributions of those hits.

In [14]:
query = """
WITH Blockbusters AS (
  SELECT * FROM video_game_sales WHERE Global_Sales > 1
)
SELECT
  Genre,
  COUNT(*) AS Num_Hits,
  ROUND(AVG(Global_Sales), 2) AS Avg_Hit_Sales
FROM Blockbusters
GROUP BY Genre
ORDER BY Num_Hits DESC;

-- Further join blockbusters with Publishers and Platforms to understand concentrations.

"""
run_query(query, "Regional Sales Summary")

## Regional Sales Summary

``````



,Genre,Num_Hits,Avg_Hit_Sales
0,Action,422,2.53
1,Sports,298,2.64
2,Shooter,248,3.09
3,Role-Playing,201,3.16
4,Platform,195,3.36
5,Racing,179,2.70
6,Misc,168,2.83
7,Fighting,122,2.22
8,Simulation,93,2.45
9,Puzzle,55,2.86



---



# What is the competitive landscape by genre? Which publishers have diversified portfolios vs. those specialized in niche genres, and how does that relate to sales efficiency?
Answer approach:
Count distinct genres each publisher covers, correlate with total sales and average sales per genre.


In [16]:
query = """
WITH PublisherPortfolio AS (
  SELECT
    Publisher,
    COUNT(DISTINCT Genre) AS Genre_Count,
    ROUND(SUM(Global_Sales), 2) AS Total_Sales
  FROM video_game_sales
  GROUP BY Publisher
)
SELECT
  Publisher,
  Genre_Count,
  Total_Sales,
  ROUND(Total_Sales / Genre_Count, 2) AS Sales_Per_Genre
FROM PublisherPortfolio
ORDER BY Total_Sales DESC;

"""
run_query(query, "Regional Sales Summary")

## Regional Sales Summary

``````



,Publisher,Genre_Count,Total_Sales,Sales_Per_Genre
0,Nintendo,12,1786.56,148.88
1,Electronic Arts,12,1110.32,92.53
2,Activision,12,727.46,60.62
3,Sony Computer Entertainment,12,607.50,50.63
4,Ubisoft,12,474.72,39.56
5,Take-Two Interactive,12,399.54,33.30
6,THQ,12,340.77,28.40
7,Konami Digital Entertainment,12,283.64,23.64
8,Sega,12,272.99,22.75
9,Namco Bandai Games,12,254.09,21.17



---



The issue in the current query is that every publisher shows the same Genre_Count value of 12, likely the total number of genres in the dataset, which skews the Sales_Per_Genre metric and doesn’t reflect actual publisher diversification.

A better way to examine this competitive landscape is to consider both the diversity (number of distinct genres a publisher publishes in) and how concentrated their sales are by genre.

Improved Approach:
Calculate total sales per publisher.

Count distinct genres each publisher covers.

Calculate average sales per genre only in which the publisher has sales, but adjusted by weighting the sales by genre share so that publishers with concentrated sales don't get artificially boosted average per genre.

Additionally, calculate a sales concentration metric, like the Herfindahl-Hirschman Index (HHI), for genre sales distribution per publisher, to quantify focus vs diversification.

In [17]:
query = """
WITH PublisherGenreSales AS (
  SELECT
    Publisher,
    Genre,
    SUM(Global_Sales) AS Genre_Sales
  FROM video_game_sales
  GROUP BY Publisher, Genre
),
PublisherStats AS (
  SELECT
    Publisher,
    COUNT(DISTINCT Genre) AS Distinct_Genres,
    SUM(Genre_Sales) AS Total_Sales,
    AVG(Genre_Sales) AS Avg_Sales_Per_Genre
  FROM PublisherGenreSales
  GROUP BY Publisher
),
PublisherHHI AS (
  SELECT
    Publisher,
    SUM(POWER(Genre_Sales / Total_Sales, 2)) AS Sales_Concentration_HHI
  FROM (
    SELECT 
      pgs.Publisher,
      pgs.Genre,
      pgs.Genre_Sales,
      ps.Total_Sales
    FROM PublisherGenreSales pgs
    JOIN (
      SELECT Publisher, SUM(Genre_Sales) AS Total_Sales
      FROM PublisherGenreSales
      GROUP BY Publisher
    ) ps ON pgs.Publisher = ps.Publisher
  )
  GROUP BY Publisher
)
SELECT 
  ps.Publisher,
  ps.Distinct_Genres,
  ROUND(ps.Total_Sales, 2) AS Total_Sales,
  ROUND(ps.Avg_Sales_Per_Genre, 2) AS Avg_Sales_Per_Genre,
  ROUND(hhi.Sales_Concentration_HHI, 3) AS Sales_Concentration_HHI
FROM PublisherStats ps
JOIN PublisherHHI hhi ON ps.Publisher = hhi.Publisher
ORDER BY Total_Sales DESC;

"""
run_query(query, "Regional Sales Summary")

## Regional Sales Summary

``````



,Publisher,Distinct_Genres,Total_Sales,Avg_Sales_Per_Genre,Sales_Concentration_HHI
0,Nintendo,12,1786.56,148.88,0.130
1,Electronic Arts,12,1110.32,92.53,0.244
2,Activision,12,727.46,60.62,0.238
3,Sony Computer Entertainment,12,607.50,50.63,0.131
4,Ubisoft,12,474.72,39.56,0.172
5,Take-Two Interactive,12,399.54,33.30,0.339
6,THQ,12,340.77,28.40,0.156
7,Konami Digital Entertainment,12,283.64,23.64,0.211
8,Sega,12,272.99,22.75,0.156
9,Namco Bandai Games,12,254.09,21.17,0.150



---



The improved query results are much better and informative now:

The Distinct_Genres is correctly 12 for all because our dataset contains 12 genres total, and these publishers appear to cover all (or almost all) genres.

The Total_Sales and Avg_Sales_Per_Genre give clear revenue scale and average earnings per genre.

The Sales_Concentration_HHI metric provides new insight:

Publishers with lower HHI (~0.13) like Nintendo and Sony have more evenly distributed sales across genres, indicating broader and balanced portfolios.

Higher HHI (e.g., Take Two Interactive at ~0.34) indicates concentration of sales in fewer genres, showing specialization or market power in key segments.

